# Description

This notebook can be used to test the implementation of the elastic energy, the elastic forces, and the force differentials.

# Load libraries

In [ ]:
import numpy as np
import igl
import meshplot as mp
import time
import torch

import sys as _sys
_sys.path.append("../src")
from create_vol_cube import *
from elastic_energy import *
from fem_system import *
from shape_optimizer import *
from utils import *
from vis_utils import *
from matplotlib import gridspec
import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float64)

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = torch.tensor(
    [[1.0,  0.0, 0.0],
     [0.0,  0.0, 1.0],
     [0.0, -1.0, 0.0]]
)

# Load mesh

Several meshes are available for you to play with under `data/`: `ball.obj`, `dinosaur.obj`, and `beam.obj`.

In [ ]:
v, _, _, t, _, _ = igl.read_obj("../data/beam.obj")
v, t = create_vol_cube(5, 3, 3)
v = torch.tensor(v)
t = torch.tensor(t)

aabb = torch.max(v, dim=0).values - torch.min(v, dim=0).values
length_scale = torch.mean(aabb)

# bv = np.unique(igl.boundary_facets(to_numpy(t))).astype(np.int64)
bv = torch.argwhere(
    torch.logical_or(
        torch.min(torch.abs(v - torch.max(v, dim=0).values.reshape(1, -1)), dim=1).values < 1.0e-4,
        torch.min(torch.abs(v - torch.min(v, dim=0).values.reshape(1, -1)), dim=1).values < 1.0e-4
    )
).reshape(-1,)
vt_surf = v[bv, :].clone()
e = igl.edges(to_numpy(t))
iv = np.array([idx for idx in range(v.shape[0]) if not idx in bv]).astype(np.int64)
be = np.array([edge for edge in e if (edge[0] in bv and edge[1] in bv)])
convertBV = {bv[i].item():i for i in range(bv.shape[0])}
beTarget = np.array([[convertBV[bEdge[0]], convertBV[bEdge[1]]] for bEdge in be])

# Linear/Non-Linear Elastic Solid

## Instantiation

We first specify the elasticity model to use for the elastic solid, as well as pinned vertices, and volumetric forces.

In [ ]:
rho     = 131.0  # [kg.m-3]
young   = 1.0e7 # [Pa] 
poisson = 0.2
force_mass = torch.zeros(size=(3,))
force_mass[2] = - 9.81

minX    = torch.min(v[:, 0])
pin_idx = torch.arange(v.shape[0])[v[:, 0] < minX + 0.2*aabb[0]].tolist()

# ee    = LinearElasticEnergy(young, poisson)
ee    = NeoHookeanElasticEnergy(young, poisson)
solid = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)


# Validate Elastic Energy Gradient (Elastic Forces) using Finite Differences on Elastic Energy

In [ ]:
fd_validation_elastic(solid, v.clone())

# Validate External Energy Gradient (External Forces) using Finite Differences on External Energy

In [ ]:
maxX        = torch.max(v[:, 0])
f_point_idx = torch.arange(v.shape[0])[v[:, 0] > maxX - 0.01*aabb[0]]

f_point = torch.zeros(size=(f_point_idx.shape[0], 3))
f_point[:, 2] = -5.0e3
f_point[:, 2] = 0.0

solid.add_point_load(f_point_idx, f_point)
fd_validation_ext(solid, v.clone())

# Validate Elastic Energy Force Differentials (Elastic Forces) using Finite Differences on Elastic Forces

In [ ]:
fd_validation_elastic_differentials(solid, v.clone())